In [1]:
import pandas as pd
import cplex
import optlang
from cplex import Cplex
from optlang.cplex_interface import Model
from optlang import Model, Variable, Constraint, Objective
import math
import json
import sys
import os
import cProfile
from optlang.symbolics import Zero, add
from IPython.core.display import HTML
from configparser import ConfigParser
config = ConfigParser()
os.chdir('/Users/ffoflonker/Documents/Command_line/Notebooks/Synbio/ecoli_staph_coculture')
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)

In [2]:
cplex_problem = Cplex()
cplex_problem.read("baseproblem.lp")
model = Model(problem=cplex_problem)


In [3]:
#data import

Ps_df = pd.read_csv ('PhenotypeStoichiometry.csv')
Ps_df = Ps_df.reset_index()
Ps_df.drop("rxn name", axis=1, inplace=True)
Ps_df.drop("index", axis=1, inplace=True)
Ps_df.set_index("rxn", inplace=True)
#print(Ps_df)
OD_df=pd.read_table('OD_pf-ec_4-29-22.txt')
OD_df=OD_df.iloc[1::2]
OD_df.drop("Plate", axis=1, inplace=True)
OD_df.drop("Cycle", axis=1, inplace=True)
OD_df.set_index("Well", inplace=True)
#print(OD_df)
RFP_df=pd.read_table('RFP_pf-ec_4-29-22.txt')
RFP_df=RFP_df.iloc[1::2]
RFP_df.drop("Plate", axis=1, inplace=True)
RFP_df.drop("Cycle", axis=1, inplace=True)
RFP_df.set_index("Well", inplace=True)
#print(RFP_df)
GFP_df=pd.read_table('GFP_pf-ec_4-29-22.txt')
GFP_df=GFP_df.iloc[1::2]
GFP_df.drop("Plate", axis=1, inplace=True)
GFP_df.drop("Cycle", axis=1, inplace=True)
GFP_df.set_index("Well", inplace=True)
#GFP_df

In [4]:
#create variable object in optlang

variables = {
    "rfbc":{}, #Coefficient for converting RFP signal values into biomass abundances
    "gfbc":{}, #Coefficient for converting GFP signal values into biomass abundances
    "odc":{}, #Coefficient for converting OD signal values into biomass abundances
    "rfpb":{}, #Biomass computed from RFP signal values at time t for experiment j
    "rfpe":{}, #Difference between model predicted biomass and rpf-based biomass at time t for experiment j
    "gfpb":{}, #Biomass computed from GFP signal values at time t for experiment j
    "gfpe":{}, #Difference between model predicted biomass and gfp-based biomass at time t for experiment j
    "odb":{}, #Biomass computed from OD signal values at time t for experiment j
    "ode":{}, #Difference between model predicted biomass and od-based biomass at time t for experiment j
    "b":{}, #Model-based biomass abundance of strain k at time t for experiment j
    "g":{}, #Growth rate of strain k at time t for experiment j
    "cvt":{}, #Rate of conversion of strain k to the stationary phase for each species
    "cvf":{}, #Rate of conversion of strain k from the stationary phase for each species. 
    "c":{}, #Concentration of metabolite I at time t for experiment j
}
constraints = {
    "rfpbc":{}, #Setting value of rfp-based biomass abundance based on rfp signal
    "rfpec":{}, #Setting value for the difference between the rfp-based biomass abundance and the model predicted rfp-biomass abundance
    "gfpbc":{}, #Setting value of gfp-based biomass abundance based on gfp signal
    "gfpec":{}, #Setting value for the different between the gfp-based biomass abundance and the model predicted gfp-biomass abundance
    "odbc":{}, #Setting value of od-based biomass abundance based on od signal
    "odec":{}, #Setting value for the different between the OD-based biomass abundance and the model predicted OD-biomass abundance
    "dbc":{}, #Iterating biomass value for each strain at each time point based on strain growth rate and transition of strains to or from other phenotypes (cv term).
    "gc":{}, #Setting the growth rate of each strain at each time step in each experiment
    "cvc":{}, #Setting an upper limit on what fraction of biomass can convert to the stationary phase in a single time step
    "dcc":{}, #Iterating metabolite concentration for each metabolite at each time point based on strain growth rate

}



In [5]:
#create variable and constraint names

strains=["ecoli_malt","ecoli_acetate","pf_acetate","pf_acetate","pf_4HB","ecoli_stationary", "pf_stationary"]
const_list=[]

#coefficients
var1=Variable("C_rfbc", lb=0,ub=1000)
variables["rfbc"]["C_rfbc"]=var1
var1=Variable("C_gfbc", lb=0,ub=1000)
variables["gfbc"]["C_gfbc"]=var1
var1=Variable("C_odbc", lb=0,ub=1000)
variables["odc"]["C_odc"]=var1


#col=time(t) index=experiment(j)

for index in RFP_df.index:
    for col in RFP_df:
        x="rfpb_"+col+"_"+index
        y="rfpe_"+col+"_"+index
        var1=Variable(x, lb=0,ub=1000)
        var2=Variable(y, lb=-100,ub=100)
        variables["rfpb"][x]=var1
        variables["rfpe"][y]=var2

        
        a="rfpbc_"+col+"_"+index
        b="rfpec_"+col+"_"+index
        constraints["rfpbc"][a]=a
        constraints["rfpec"][b]=b
for index in GFP_df.index:
    for col in GFP_df:
        x="gfpb_"+col+"_"+index
        y="gfpe_"+col+"_"+index
        var1=Variable(x, lb=0,ub=1000)
        var2=Variable(y, lb=-100,ub=100)
        variables["gfpb"][x]=var1
        variables["gfpe"][y]=var2
        
        a="gfpbc_"+col+"_"+index
        b="gfpec_"+col+"_"+index
        constraints["gfpbc"][a]=a
        constraints["gfpec"][b]=b       
for index in OD_df.index:
    for col in OD_df:
        x="odb_"+col+"_"+index
        y="ode_"+col+"_"+index
        var1=Variable(x, lb=0,ub=1000)
        var2=Variable(y, lb=-100,ub=100)       
        variables["odb"][x]=var1
        variables["ode"][y]=var2
        
        a="odbc_"+col+"_"+index
        b="odec_"+col+"_"+index
        constraints["odbc"][a]=a
        constraints["odec"][b]=b
        
        for strain in strains:
            z="b_"+strain+ "_"+col+"_"+index
            var1=Variable(z, lb=0,ub=1000)
            variables["b"][z]=var1
            a="g_"+strain+ "_"+col+"_"+index
            var2=Variable(a, lb=0,ub=1000)
            variables["g"][a]=var2
            b="cvt_"+strain+ "_"+col+"_"+index
            var3=Variable(b, lb=0,ub=100)
            variables["cvt"][b]=var3
            c="cvf_"+strain+ "_"+col+"_"+index
            var4=Variable(c, lb=0,ub=100)
            variables["cvf"][c]=var4
            
            d="dbc_"+strain+ "_"+col+"_"+index
            constraints["dbc"][d]=d
            e="gc_"+strain+ "_"+col+"_"+index
            constraints["gc"][e]=e
            f="cvc_"+strain+ "_"+col+"_"+index
            constraints["cvc"][f]=f
            
        for met in Ps_df.index:
            for pheno in Ps_df:
                x="c_"+met+"_"+col+"_"+index
                var1=Variable(x, lb=0,ub=1000)
                variables["c"][x]=var1
                y="dcc_"+met+"_"+col+"_"+index
                constraints["dcc"][y]=y


            


In [6]:
parameters ={
    
    "rfps":{"ecoli_malt":0,"ecoli_acetate":0,"ecoli_stationary":0,"pf_acetate":1,"pf_4HB":1,"pf_stationary":1}, #Set to one if strain k is Pseudomonas and zero otherwise
    "gfps":{"ecoli_malt":1,"ecoli_acetate":1,"ecoli_stationary":1,"pf_acetate":0,"pf_4HB":0,"pf_stationary":0}, #Set to one if strain k is E. coli and zero otherwise
    #iteratively tuned
    "dt":{"dt":600}, # Size of time step used in simulation in seconds
    "cvct":{"cvct":1}, #Coefficient for the minimization of phenotype conversion to the stationary phase. 
    "cvcf":{"cvcf":1}, #Coefficient for the minimization of phenotype conversion from the stationary phase. 
    "bcv":{"bcv":1}, #This is the highest fraction of biomass for a given strain that can change phenotypes in a single time step
    "cvmin":{"cvmin":0.1}, #This is the lowest value the limit on phenotype conversion goes, 
    #from experimental
    "rfp":{}, #Measured RFP abundance at time t for experiment j
    "gfp":{}, #Measured GFP abundance at time t for experiment j 
    "od":{}, # Measured OD at time t for experiment j
    #need to code
    "y":{}, #Stoichiometry for interaction of strain k with metabolite i
    "v":{}
    

}

#input parameters
for index in RFP_df.index:
    for col in RFP_df:
        a="rfp_"+col+"_"+index
        parameters["rfp"][a]= RFP_df.loc[index,col]
        b="gfp_"+col+"_"+index
        parameters["gfp"][b]= GFP_df.loc[index,col]
        c="od_"+col+"_"+index
        parameters["od"][c]= OD_df.loc[index,col]
        
        for strain in strains:
            z="v_"+strain+"_"+col+"_"+index
            parameters["v"][z]=0
        
for met in Ps_df.index:
    for pheno in Ps_df:
        d="PS_"+pheno+"_"+met
        parameters["y"][d]= Ps_df.loc[met,pheno]

        
    
        
        

In [7]:
const_list=[]
for index in RFP_df.index:
    for col in RFP_df:
        x="rfpb_"+col+"_"+index
        a="rfp_"+col+"_"+index
        const = Constraint(
                variables["rfpb"][x]/ variables["rfbc"]["C_rfbc"],
                ub=parameters["rfp"][a],
                lb=parameters["rfp"][a],
                name="rfpbc_"+col+"_"+index,
            )
        const_list.append(const)
        
        x="rfpe_"+col+"_"+index
        y="rfpb_"+col+"_"+index
        to_sum=[]
        for strain in strains:
            z="b_"+strain+ "_"+col+"_"+index
            to_sum.append(z)
        const = Constraint(
                (variables["rfpb"][y] -variables["rfpe"][x])/sum(variables["b"][item] for item in to_sum),
                ub=3, #does this look right?
                lb=3,
                name="rfpec_"+col+"_"+index,
            )
        const_list.append(const)
for index in GFP_df.index:
    for col in GFP_df:
        x="gfpb_"+col+"_"+index
        a="gfp_"+col+"_"+index
        const = Constraint(
                variables["gfpb"][x]/ variables["gfbc"]["C_gfbc"],
                ub=parameters["gfp"][a],
                lb=parameters["gfp"][a],
                name="gfpbc_"+col+"_"+index,
            )
        const_list.append(const)
        
        x="gfpe_"+col+"_"+index
        y="gfpb_"+col+"_"+index
        to_sum=[]
        for strain in strains:
            z="b_"+strain+ "_"+col+"_"+index
            to_sum.append(z)
        const = Constraint(
                (variables["gfpb"][y] -variables["gfpe"][x])/sum(variables["b"][item] for item in to_sum),
                ub=3, #does this look right?
                lb=3,
                name="gfpec_"+col+"_"+index,
            )
        const_list.append(const)
for index in OD_df.index:
    for col in OD_df:
        x="odb_"+col+"_"+index
        a="od_"+col+"_"+index
        const = Constraint(
                variables["odb"][x]/ variables["odc"]["C_odc"],
                ub=parameters["od"][a],
                lb=parameters["od"][a],
                name="odbc_"+col+"_"+index,
            )
        const_list.append(const)
        
        x="ode_"+col+"_"+index
        y="odb_"+col+"_"+index
        to_sum=[]
        for strain in strains:
            z="b_"+strain+ "_"+col+"_"+index
            to_sum.append(z)
        const = Constraint(
                (variables["odb"][y] -variables["ode"][x])/sum(variables["b"][item] for item in to_sum),
                ub=1, #what to set this to?
                lb=1,
                name="odec_"+col+"_"+index,
            )
        const_list.append(const)
        
        for strain in strains:
            a="g_"+strain+ "_"+col+"_"+index
            b="b_"+strain+ "_"+col+"_"+index
            c="v_"+strain+ "_"+col+"_"+index
            const = Constraint(
                    variables["g"][a]/ variables["b"][b],
                    ub=parameters["v"][c],
                    lb=parameters["v"][c],
                    name="gc_"+strain+ "_"+col+"_"+index,
                )
            const_list.append(const)
            
            b="b_"+strain+ "_"+col+"_"+index
            c="cvt_"+strain+ "_"+col+"_"+index
            const = Constraint(
                    variables["cvt"][c]-(parameters["bcv"]["bcv"]*variables["b"][b]),
                    ub=parameters["cvmin"]["cvmin"],
                    lb=0, # does this make sense?
                    name="cvc_"+strain+ "_"+col+"_"+index,
                )
            const_list.append(const)

############################## neeed to fix the t+1 issue for this section to work

            if "stationary" in strain: ##### not done coding this constraint
                b1="b1_"+strain+ "_"+col+"_"+index
                col2= int(col)+1
                b2="b_"+strain+"_"+str(col2)+"_"+index # need to do something about the variables that are out of time range
                c="cvt_"+strain+ "_"+col+"_"+index
                f="cvf_"+strain+ "_"+col+"_"+index
                const = Constraint(
                        variables["b"][b2]-variables["b"][b1],
                        ub=,
                        lb=, 
                        name="dbc_"+strain+ "_"+col+"_"+index,
                    )
                const_list.append(const)
                
#             else:
#                 b1="b1_"+strain+ "_"+col+"_"+index
#                 col2= int(col)+1
#                 b2="b_"+strain+"_"+str(col2)+"_"+index # need to do something about the variables that are out of time range
#                 c="cvt_"+strain+ "_"+col+"_"+index
#                 f="cvf_"+strain+ "_"+col+"_"+index
#                 g="g_"+strain+ "_"+col+"_"+index
#                 const = Constraint(
#                         (variables["b"][b2]-variables["b"][b1]-variables["cvf"][f] + variables["cvt"][c])/variables["g"][g],
#                         ub= parameters["dt"]["dt"],
#                         lb= parameters["dt"]["dt"], 
#                         name="dbc_"+strain+ "_"+col+"_"+index,
#                     )
#                 const_list.append(const)
            
#             for met in Ps_df.index:
#                 for pheno in Ps_df:
#                     to_sum=[]
                    
#                     c1= "c_"+met+"_"+col+"_"+index
#                     col2= int(col)+1
#                     c2= "c_"+met+"_"+str(col2)+"_"+index # need to do something about the variables that are out of time range
#                     for strain in strains:
#                         g="g_"+strain+ "_"+col+"_"+index
#                         y="PS_"+strain+"_"+met
#                         z=variables["g"][g]*parameters["y"][y]
#                         to_sum.append(z)
#                     const = Constraint(
#                             (variables["c"][c2]-variables["c"][c1])/(sum(items for items in to_sum)),
#                             ub= parameters["dt"]["dt"],
#                             lb= parameters["dt"]["dt"],
#                             name="dcc_"+met+"_"+col+"_"+index,
#                         )
#                     const_list.append(const)


model.add(const_list)

KeyError: 'c_bio_167_B2'

In [ ]:
# Define the objective
to_sum1=[]
for index in GFP_df.index:
    for col in GFP_df:
        x="gfpe_"+col+"_"+index
        to_sum1.append(x)
gfpe_obj=sum(items**2 for items in to_sum1)

to_sum1=[]
for index in RFP_df.index:
    for col in RFP_df:
        x="rfpe_"+col+"_"+index
        to_sum1.append(x)
rfpe_obj= sum(items**2 for items in to_sum1)

to_sum1=[]
to_sum2=[]
to_sum3=[]
for index in OD_df.index:
    for col in OD_df:
        x="ode_"+col+"_"+index
        to_sum1.append(x)
        for strain in strains:
            y="cvt_"+strain+ "_"+col+"_"+index
            z="cvf_"+strain+ "_"+col+"_"+index
            a=parameters["cvct"]["cvct"] * y
            b=parameters["cvcf"]["cvcf"] * z
            to_sum2.append(a)
            to_sum3.append(b)
ode_obj= sum(items**2 for items in to_sum1)
cvct_cvt_obj=sum(items for items in to_sum2)
cvcf_cvf_obj=sum(items for items in to_sum3)

obj = Objective(
     gfpe_obj + rfpe_obj + ode_obj - cvct_cvt_obj - cvcf_cvf_obj,
    direction="min"
)

In [ ]:
sol=model.optimize()

In [19]:
#print solution
f = open("solution.txt", "w")
for parameter in parameters:
    print(parameter)
    for item in parameters[parameter]:
        print(item)
        name="parameters_"+parameter+"_"+item+":"
        f.write(name, parameters[parameter][item])
f.close()

rfps
ecoli_malt


TypeError: write() takes exactly one argument (2 given)